# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fed6cd4a100>
├── 'a' --> tensor([[ 0.9889, -0.3573,  0.5165],
│                   [ 1.5832, -0.6554, -0.5770]])
└── 'x' --> <FastTreeValue 0x7fed6cd4a280>
    └── 'c' --> tensor([[-0.1608,  0.0706, -0.1903,  0.0179],
                        [ 0.1710, -0.3235, -1.7786, -1.3428],
                        [-2.3486, -0.1342,  0.2478, -0.5605]])

In [4]:
t.a

tensor([[ 0.9889, -0.3573,  0.5165],
        [ 1.5832, -0.6554, -0.5770]])

In [5]:
%timeit t.a

65.2 ns ± 0.0672 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fed6cd4a100>
├── 'a' --> tensor([[-1.0206,  0.0066,  0.2278],
│                   [ 1.8044,  1.2723,  1.1998]])
└── 'x' --> <FastTreeValue 0x7fed6cd4a280>
    └── 'c' --> tensor([[-0.1608,  0.0706, -0.1903,  0.0179],
                        [ 0.1710, -0.3235, -1.7786, -1.3428],
                        [-2.3486, -0.1342,  0.2478, -0.5605]])

In [7]:
%timeit t.a = new_value

68.3 ns ± 0.135 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.9889, -0.3573,  0.5165],
               [ 1.5832, -0.6554, -0.5770]]),
    x: Batch(
           c: tensor([[-0.1608,  0.0706, -0.1903,  0.0179],
                      [ 0.1710, -0.3235, -1.7786, -1.3428],
                      [-2.3486, -0.1342,  0.2478, -0.5605]]),
       ),
)

In [10]:
b.a

tensor([[ 0.9889, -0.3573,  0.5165],
        [ 1.5832, -0.6554, -0.5770]])

In [11]:
%timeit b.a

59.1 ns ± 0.0618 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.9954,  0.0837, -0.0721],
               [-2.3077, -1.7276, -0.9639]]),
    x: Batch(
           c: tensor([[-0.1608,  0.0706, -0.1903,  0.0179],
                      [ 0.1710, -0.3235, -1.7786, -1.3428],
                      [-2.3486, -0.1342,  0.2478, -0.5605]]),
       ),
)

In [13]:
%timeit b.a = new_value

504 ns ± 0.112 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

820 ns ± 0.926 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.9 µs ± 22.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

143 µs ± 361 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

141 µs ± 334 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fecb5615820>
├── 'a' --> tensor([[[ 0.9889, -0.3573,  0.5165],
│                    [ 1.5832, -0.6554, -0.5770]],
│           
│                   [[ 0.9889, -0.3573,  0.5165],
│                    [ 1.5832, -0.6554, -0.5770]],
│           
│                   [[ 0.9889, -0.3573,  0.5165],
│                    [ 1.5832, -0.6554, -0.5770]],
│           
│                   [[ 0.9889, -0.3573,  0.5165],
│                    [ 1.5832, -0.6554, -0.5770]],
│           
│                   [[ 0.9889, -0.3573,  0.5165],
│                    [ 1.5832, -0.6554, -0.5770]],
│           
│                   [[ 0.9889, -0.3573,  0.5165],
│                    [ 1.5832, -0.6554, -0.5770]],
│           
│                   [[ 0.9889, -0.3573,  0.5165],
│                    [ 1.5832, -0.6554, -0.5770]],
│           
│                   [[ 0.9889, -0.3573,  0.5165],
│                    [ 1.5832, -0.6554, -0.5770]]])
└── 'x' --> <FastTreeValue 0x7fed402e7ca0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33 µs ± 64.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fed6cd151f0>
├── 'a' --> tensor([[ 0.9889, -0.3573,  0.5165],
│                   [ 1.5832, -0.6554, -0.5770],
│                   [ 0.9889, -0.3573,  0.5165],
│                   [ 1.5832, -0.6554, -0.5770],
│                   [ 0.9889, -0.3573,  0.5165],
│                   [ 1.5832, -0.6554, -0.5770],
│                   [ 0.9889, -0.3573,  0.5165],
│                   [ 1.5832, -0.6554, -0.5770],
│                   [ 0.9889, -0.3573,  0.5165],
│                   [ 1.5832, -0.6554, -0.5770],
│                   [ 0.9889, -0.3573,  0.5165],
│                   [ 1.5832, -0.6554, -0.5770],
│                   [ 0.9889, -0.3573,  0.5165],
│                   [ 1.5832, -0.6554, -0.5770],
│                   [ 0.9889, -0.3573,  0.5165],
│                   [ 1.5832, -0.6554, -0.5770]])
└── 'x' --> <FastTreeValue 0x7fed6cd7adf0>
    └── 'c' --> tensor([[-0.1608,  0.0706, -0.1903,  0.0179],
                        [ 0.1710, -0.3235, -1.7786, -1.3428],
                 

In [23]:
%timeit t_cat(trees)

30.9 µs ± 66.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.7 µs ± 64.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.9889, -0.3573,  0.5165],
                [ 1.5832, -0.6554, -0.5770]],
       
               [[ 0.9889, -0.3573,  0.5165],
                [ 1.5832, -0.6554, -0.5770]],
       
               [[ 0.9889, -0.3573,  0.5165],
                [ 1.5832, -0.6554, -0.5770]],
       
               [[ 0.9889, -0.3573,  0.5165],
                [ 1.5832, -0.6554, -0.5770]],
       
               [[ 0.9889, -0.3573,  0.5165],
                [ 1.5832, -0.6554, -0.5770]],
       
               [[ 0.9889, -0.3573,  0.5165],
                [ 1.5832, -0.6554, -0.5770]],
       
               [[ 0.9889, -0.3573,  0.5165],
                [ 1.5832, -0.6554, -0.5770]],
       
               [[ 0.9889, -0.3573,  0.5165],
                [ 1.5832, -0.6554, -0.5770]]]),
    x: Batch(
           c: tensor([[[-0.1608,  0.0706, -0.1903,  0.0179],
                       [ 0.1710, -0.3235, -1.7786, -1.3428],
                       [-2.3486, -0.1342,  0.2478, -0.5605]],
         

In [26]:
%timeit Batch.stack(batches)

77.7 µs ± 319 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.9889, -0.3573,  0.5165],
               [ 1.5832, -0.6554, -0.5770],
               [ 0.9889, -0.3573,  0.5165],
               [ 1.5832, -0.6554, -0.5770],
               [ 0.9889, -0.3573,  0.5165],
               [ 1.5832, -0.6554, -0.5770],
               [ 0.9889, -0.3573,  0.5165],
               [ 1.5832, -0.6554, -0.5770],
               [ 0.9889, -0.3573,  0.5165],
               [ 1.5832, -0.6554, -0.5770],
               [ 0.9889, -0.3573,  0.5165],
               [ 1.5832, -0.6554, -0.5770],
               [ 0.9889, -0.3573,  0.5165],
               [ 1.5832, -0.6554, -0.5770],
               [ 0.9889, -0.3573,  0.5165],
               [ 1.5832, -0.6554, -0.5770]]),
    x: Batch(
           c: tensor([[-0.1608,  0.0706, -0.1903,  0.0179],
                      [ 0.1710, -0.3235, -1.7786, -1.3428],
                      [-2.3486, -0.1342,  0.2478, -0.5605],
                      [-0.1608,  0.0706, -0.1903,  0.0179],
                      [ 0.1710, -

In [28]:
%timeit Batch.cat(batches)

164 µs ± 164 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

328 µs ± 2.25 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
